In [16]:
import pandas as pd

**Third country nationals refused entry at the external borders - annual data (rounded)**

---

Units of measure are absolute numbers of persons.

In [17]:
df = pd.read_csv('../../data/migrants_refused_entry_at_the_border_migr_eirf.csv')

In [18]:
df.head()

,DATAFLOW,LAST UPDATE,freq,border,unit,reason,citizen,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:MIGR_EIRFS$DEFAULTVIEW(1.0),NaN,Annual,Total,Person,Total,Total,Austria,2014,455,NaN,NaN
1,ESTAT:MIGR_EIRFS$DEFAULTVIEW(1.0),NaN,Annual,Total,Person,Total,Total,Austria,2015,560,NaN,NaN
2,ESTAT:MIGR_EIRFS$DEFAULTVIEW(1.0),NaN,Annual,Total,Person,Total,Total,Austria,2016,460,NaN,NaN
3,ESTAT:MIGR_EIRFS$DEFAULTVIEW(1.0),NaN,Annual,Total,Person,Total,Total,Austria,2017,740,NaN,NaN
4,ESTAT:MIGR_EIRFS$DEFAULTVIEW(1.0),NaN,Annual,Total,Person,Total,Total,Austria,2018,400,NaN,NaN


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     327 non-null    object 
 1   LAST UPDATE  0 non-null      float64
 2   freq         327 non-null    object 
 3   border       327 non-null    object 
 4   unit         327 non-null    object 
 5   reason       327 non-null    object 
 6   citizen      327 non-null    object 
 7   geo          327 non-null    object 
 8   TIME_PERIOD  327 non-null    int64  
 9   OBS_VALUE    327 non-null    int64  
 10  OBS_FLAG     2 non-null      object 
 11  CONF_STATUS  0 non-null      float64
dtypes: float64(2), int64(2), object(8)
memory usage: 30.8+ KB


In [20]:
df.duplicated().sum()

np.int64(0)

In [21]:
df.isnull().sum()

DATAFLOW         0
LAST UPDATE    327
freq             0
border           0
unit             0
reason           0
citizen          0
geo              0
TIME_PERIOD      0
OBS_VALUE        0
OBS_FLAG       325
CONF_STATUS    327
dtype: int64

In [22]:
df = df.drop(columns=["DATAFLOW", "LAST UPDATE", "OBS_FLAG", "CONF_STATUS"])

In [23]:
for col in df.columns:
    print(f"Uniques values in {col}: {df[col].unique()}")

Uniques values in freq: ['Annual']
Uniques values in border: ['Total']
Uniques values in unit: ['Person']
Uniques values in reason: ['Total']
Uniques values in citizen: ['Total']
Uniques values in geo: ['Austria' 'Belgium' 'Bulgaria' 'Switzerland' 'Cyprus' 'Czechia' 'Germany'
 'Denmark' 'Estonia' 'Greece' 'Spain'
 'European Union - 27 countries (from 2020)' 'Finland' 'France' 'Croatia'
 'Hungary' 'Ireland' 'Iceland' 'Italy' 'Liechtenstein' 'Lithuania'
 'Luxembourg' 'Latvia' 'Montenegro' 'Malta' 'Netherlands' 'Norway'
 'Poland' 'Portugal' 'Romania' 'Sweden' 'Slovenia' 'Slovakia'
 'United Kingdom']
Uniques values in TIME_PERIOD: [2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]
Uniques values in OBS_VALUE: [   455    560    460    740    400    770    310    555    575    680
   1535   1640   1530   2440   2200   2300   1140   1720   1805   2315
   2405   2170   2870   3220   4245   4685   6180   7955   7230    920
    945    900   1225   1205   1165   1305   2100   1630   1295    425


In [24]:
df = df[df["geo"] != "European Union - 27 countries (from 2020)"]

In [25]:
df = df.drop(columns=["border", "citizen", "unit", "freq"])
df = df.dropna()

In [26]:
df = df.rename(columns={"OBS_VALUE": "Migrants number"})

In [27]:
df.head()

,reason,geo,TIME_PERIOD,Migrants number
0,Total,Austria,2014,455
1,Total,Austria,2015,560
2,Total,Austria,2016,460
3,Total,Austria,2017,740
4,Total,Austria,2018,400


In [28]:
population_df = pd.read_csv('../../data/population_data_tps00001.csv')
population_df = population_df.drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "indic_de", "OBS_FLAG", "CONF_STATUS"])
population_df = population_df.rename(columns={"OBS_VALUE": "Population"})
merged_df = df.merge(population_df, how="left", left_on=["TIME_PERIOD", "geo"], right_on=["TIME_PERIOD", "geo"])

In [29]:
merged_df["Migrants number per 100k inhabitants"] = round(
    merged_df["Migrants number"] / merged_df["Population"] * 100_000, 2)

In [30]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   reason                                317 non-null    object 
 1   geo                                   317 non-null    object 
 2   TIME_PERIOD                           317 non-null    int64  
 3   Migrants number                       317 non-null    int64  
 4   Population                            317 non-null    int64  
 5   Migrants number per 100k inhabitants  317 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 15.0+ KB


In [31]:
merged_df = merged_df.drop(columns=["Population"])

In [32]:
merged_df.to_csv("../../processed_data/migrants_refused_entry_ar_the_border.csv", index=False)